### 약 이름 뽑아오기

In [196]:
import datetime
import codecs
import urllib
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

def cur_timestamp(): # 현재 시간을 타임스탬프로 만들어주는 함수
    now = datetime.datetime.now()
    timestamp = int(now.timestamp()*1000)
    return str(timestamp)

def convert_to_utf8(string): # 한글을 url에 입력하게 변경해줌
    return urllib.parse.quote(string)

def get_response(url):
    headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Cookie': 'ASPSESSIONIDQQCTSAQD=DPKPMKCCGAOKCEILBBICOKFF; _gid=GA1.2.68717777.1701659617; _ga_D05L6BYNX0=GS1.1.1701659616.1.1.1701660707.60.0.0; _ga=GA1.1.1789168813.1701659617',
    'Host': 'www.health.kr',
    'Referer': 'https://www.health.kr/interaction/food.asp',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"'}
    response = requests.get(url, headers=headers)
    return json.loads(response.text)

def search_med(name):
    time = cur_timestamp()
    search_name = convert_to_utf8(name)
    url = f'https://www.health.kr/interaction/ajax/ajax_food_list.asp?search_word={search_name}&_={time}'
    resp = get_response(url)
    return pd.DataFrame(resp)
# search_med('비아그라') - dcode, dname, dename, dtype, dsunb,info_count, ingr_mg

def search_ingd(dcode):
    # target : https://www.health.kr/interaction/ajax/ajax_food_ingd.asp?drug_code=A11ABBBBB2544&_=1701671995366'
    time = cur_timestamp()
    url = f'https://www.health.kr/interaction/ajax/ajax_food_ingd.asp?drug_code={dcode}&_={time}'
    resp = get_response(url)
    return pd.DataFrame(resp)
# search_ingd('A11ABBBBB2544') : ingd_code, ingd_name


def search_info(ingd_code):
    time = cur_timestamp()
    url = f'https://www.health.kr/interaction/ajax/ajax_food_sunblist.asp?icode={ingd_code}&_={time}'
    resp = get_response(url)
    return pd.DataFrame(resp)


In [161]:
med_list = search_med('비아그라정')

200


In [162]:
med_list

,dcode,dname,dename,dtype,dsunb,info_count,ingr_mg
0,A11ABBBBB2544,비아그라정100mg,Viagra Tab. 100mg,D,Sildenafil Citrate 140.45mg,2,Sildenafil Citrate 140.45mg
1,A11ABBBBB0960,비아그라정25mg,Viagra Tab. 25mg,D,Sildenafil Citrate 35.112mg,2,Sildenafil Citrate 35.112mg
2,A11ABBBBB0961,비아그라정50mg,Viagra Tab. 50mg,D,Sildenafil Citrate 70.225mg,2,Sildenafil Citrate 70.225mg


In [163]:
for idx, row in med_list.iterrows():
    y1 = search_ingd(row.dcode)
    for idx, row in y1.iterrows():
        print(search_info(row.ingd_code))

200
200
                       sunb_name  \
0  실데나필시트르산염(Sildenafil Citrate)   
1  실데나필시트르산염(Sildenafil Citrate)   

                                     contents_origin  
0  식사와 관계없이 복용할 수 있어요. 하지만 고지방식이 후 이 약을 복용시, 약물발현...  
1              음주를 피하세요. 음주시 심한 현기증과 두통을 유발할 수 있어요.   
200
200
                       sunb_name  \
0  실데나필시트르산염(Sildenafil Citrate)   
1  실데나필시트르산염(Sildenafil Citrate)   

                                     contents_origin  
0  식사와 관계없이 복용할 수 있어요. 하지만 고지방식이 후 이 약을 복용시, 약물발현...  
1              음주를 피하세요. 음주시 심한 현기증과 두통을 유발할 수 있어요.   
200
200
                       sunb_name  \
0  실데나필시트르산염(Sildenafil Citrate)   
1  실데나필시트르산염(Sildenafil Citrate)   

                                     contents_origin  
0  식사와 관계없이 복용할 수 있어요. 하지만 고지방식이 후 이 약을 복용시, 약물발현...  
1              음주를 피하세요. 음주시 심한 현기증과 두통을 유발할 수 있어요.   


In [150]:
med_list = search_med('비아')

200


In [157]:
ingd = search_ingd('비아그라정100mg')


200


""


### 약이름 목록 만들기

1. 비아로 시작하는 약이름 목록 추출 df1
2. 추출한 약이름에서 성분과 내용 뽑아서 합치기 df2

In [198]:
### 1. 약이름 목록 추출

df1 = search_med('비아')

In [199]:
medlist = df1[['dcode', 'dname']].copy()
medlist.to_csv('data/med_list.csv')

In [200]:
ingd_list = pd.DataFrame()
for idx, row in medlist.iterrows():
    df = search_ingd(row.dcode)
    ingd_list = pd.concat([ingd_list, df], ignore_index= True).drop_duplicates(subset = 'ingd_code', keep = 'first')

In [201]:
ingd_list.to_csv('data/ingd_list.csv')

In [202]:
info_list = pd.DataFrame()

for idx, row in ingd_list.iloc[:20].iterrows():
    df2 = search_info(row.ingd_code)
    info_list = pd.concat([info_list, df2], ignore_index= True).drop_duplicates(subset=['sunb_name', 'contents_origin'], keep = 'first')

In [205]:
info_list.to_csv('data/info_list.csv')